In [9]:
from lean_interact import Command, LeanServer, LeanREPLConfig, TempRequireProject, ProofStep

config = LeanREPLConfig(local_repl_path = "/Users/sjm/coding/projects/Newton/repl", lean_version="v4.21.0-rc3", project=TempRequireProject("mathlib"))
print(config.__dict__)

server = LeanServer(config)


{'lean_version': 'v4.21.0-rc3', 'project': TempRequireProject(require='mathlib'), 'repl_git': 'https://github.com/augustepoiroux/repl', 'repl_rev': 'v1.0.9', 'cache_dir': PosixPath('/Users/sjm/miniconda3/lib/python3.13/site-packages/lean_interact/cache'), 'local_repl_path': PosixPath('/Users/sjm/coding/projects/Newton/repl'), 'build_repl': True, 'memory_hard_limit_mb': None, 'lake_path': PosixPath('lake'), 'verbose': False, '_timeout_lock': 300, '_stdout': -3, '_stderr': -3, '_cache_repl_dir': PosixPath('/Users/sjm/coding/projects/Newton/repl'), '_working_dir': PosixPath('/Users/sjm/miniconda3/lib/python3.13/site-packages/lean_interact/cache/tmp_projects/v4.21.0-rc3/1e05fd82f2c4fc489117f059ac17237437a6dbc201ad266e76e4f63abf7f4e88')}


In [19]:

s="""import Lean
import Mathlib
open Lean Elab Command Term

/--
`#find_in_module My.Module.Name`
Finds all theorems defined in the specified module (file).
-/
syntax (name := findInModule) "#find_in_module" ident : command

@[command_elab findInModule]
def elabFindInModule : CommandElab := fun stx => do
  runTermElabM fun _ => do
    let moduleName := stx[1].getId
    let env ← getEnv
    let some modIdx := env.getModuleIdx? moduleName
      | logError m!"Module `{moduleName}` not found. Make sure it's imported."
        return

    let thms := env.constants.toList.filter fun (declName, cinfo) =>
      let isTheorem := match cinfo with | .thmInfo _ => true | _ => false
      -- Check if the theorem's module index matches the one we found
      isTheorem && env.getModuleIdxFor? declName == some modIdx

    if thms.isEmpty then
      logInfo m!"[find_in_module] No theorems found in module `{moduleName}`."
    else
      logInfo m!"--- Theorems in module `{moduleName}` ---"
      for (name, info) in thms do
        logInfo m!"{name} : {info.type}"

-- This will now show the theorems defined in that specific file.
#find_in_module Mathlib.Algebra.Order.Group.Unbundled.Abs
"""
response = server.run(Command(cmd=s))
print(response)

CommandResponse(env=5, messages=[Message(start_pos=Pos(column=0, line=33), data='--- Theorems in module `Mathlib.Algebra.Order.Group.Unbundled.Abs` ---', severity='info', end_pos=Pos(column=57, line=33)), Message(start_pos=Pos(column=0, line=33), data='abs_eq_max_neg : ∀ {α : Type u_1} [inst : AddGroup α] [inst_1 : LinearOrder α] {a : α}, |a| = max a (-a)', severity='info', end_pos=Pos(column=57, line=33)), Message(start_pos=Pos(column=0, line=33), data='mabs_le_one._proof_3 : ∀ {α : Type u_1} [inst : Group α] [inst_1 : LinearOrder α] [MulLeftMono α] {a : α}\n  [MulRightMono α], (mabs a ≤ 1) = (a = 1)', severity='info', end_pos=Pos(column=57, line=33)), Message(start_pos=Pos(column=0, line=33), data='inv_le_mabs : ∀ {α : Type u_1} [inst : Lattice α] [inst_1 : Group α] (a : α), a⁻¹ ≤ mabs a', severity='info', end_pos=Pos(column=57, line=33)), Message(start_pos=Pos(column=0, line=33), data='le_mabs : ∀ {α : Type u_1} [inst : Group α] [inst_1 : LinearOrder α] {a b : α}, a ≤ mabs b ↔ a ≤ b

In [15]:
[data.data for data in response.messages if data.severity == 'info']

['MyTest.my_thm_1 : True',
 'MyTest.Sub.my_thm_2 : 1 = 1',
 'MyTest.my_thm_1 : True',
 'MyTest.Sub.my_thm_2 : 1 = 1']

In [4]:

response = server.run(Command(cmd="""import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat
example (a b c d e : ℝ) (h : a = b + c) (h' : b = d - e) : a + e = d + c := by
  rw[h]
  sorry
"""))
response

CommandResponse(env=2, messages=[Message(start_pos=Pos(column=0, line=7), data="declaration uses 'sorry'", severity='warning', end_pos=Pos(column=7, line=7))], sorries=[Sorry(start_pos=Pos(column=2, line=9), proof_state=2, goal="a b c d e : ℝ\nh : a = b + c\nh' : b = d - e\n⊢ b + c + e = d + c", end_pos=Pos(column=7, line=9))])

In [7]:
from lean_interact import RemoveProofState

for i in range(21, 61):
    server.run(RemoveProofState(remove_proof_state=i))

In [8]:
for i in range(1, 21):
    response = server.run(ProofStep(proof_state=0, tactic="""(
      ring
    )"""))
    print(response)

ProofStepResponse(proof_status='Incomplete: open goals remain', proof_state=81, messages=[Message(data='Try this: ring_nf', end_pos=Pos(column=0, line=0), severity='info', start_pos=Pos(column=0, line=0))], goals=["a b c d e : ℝ\nh : a = b + c\nh' : b = d - e\n⊢ b + c + e = c + d"])
ProofStepResponse(proof_status='Incomplete: open goals remain', proof_state=82, messages=[Message(data='Try this: ring_nf', end_pos=Pos(column=0, line=0), severity='info', start_pos=Pos(column=0, line=0))], goals=["a b c d e : ℝ\nh : a = b + c\nh' : b = d - e\n⊢ b + c + e = c + d"])
ProofStepResponse(proof_status='Incomplete: open goals remain', proof_state=83, messages=[Message(data='Try this: ring_nf', end_pos=Pos(column=0, line=0), severity='info', start_pos=Pos(column=0, line=0))], goals=["a b c d e : ℝ\nh : a = b + c\nh' : b = d - e\n⊢ b + c + e = c + d"])
ProofStepResponse(proof_status='Incomplete: open goals remain', proof_state=84, messages=[Message(data='Try this: ring_nf', end_pos=Pos(column=0, li

In [ ]:
response = server.run(ProofStep(proof_state=0, tactic="rw [h']"))

ProofStepResponse(proof_status='Completed', goals=[], proof_state=1)


In [28]:
response = server.run(ProofStep(proof_state=1, tactic="""(
nlinarith
)"""))
print(response)

ProofStepResponse(proof_status="Error: kernel type check failed: (kernel) declaration has metavariables '[anonymous]'", goals=[], proof_state=3)
